In [2]:
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql import Row

spark = SparkSession.builder.getOrCreate()

### Import Data into SPARK Data Frame

In [3]:
df = spark.read.csv('metadata.csv')

In [4]:
df.show()

+----------+----+-------+---------+--------------+------------+-------------+-----------+--------+------------+----------+-------+-----+-------+-------+-------+-------+-------+-------+------------+------------+------------+------------+------------+------------+------+------+------+------+------+------+
|       _c0| _c1|    _c2|      _c3|           _c4|         _c5|          _c6|        _c7|     _c8|         _c9|      _c10|   _c11| _c12|   _c13|   _c14|   _c15|   _c16|   _c17|   _c18|        _c19|        _c20|        _c21|        _c22|        _c23|        _c24|  _c25|  _c26|  _c27|  _c28|  _c29|  _c30|
+----------+----+-------+---------+--------------+------------+-------------+-----------+--------+------------+----------+-------+-----+-------+-------+-------+-------+-------+-------+------------+------------+------------+------------+------------+------------+------+------+------+------+------+------+
| Sample_ID|Year|Country|Continent|Beta.lactamase|Azithromycin|Ciprofloxacin|Ceftriax

24/11/23 18:54:40 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
